In [7]:
from playwright.async_api import async_playwright
import asyncio
import os
import pandas as pd
from pathlib import Path
import openpyxl
import tabula

In [30]:
# Určení proměnných
# url = odkaz na stránky se stažením souborů
# download_path = cesta kam se mají uložit

url = "https://mpo.gov.cz/cz/podnikani/zivnostenske-podnikani/statisticke-udaje-o-podnikatelich/pocty-podnikatelu-dle-obcanstvi-podnikajicich-v-ceske-republice--151024/"
download_path = "./Download"

In [23]:
# Definujeme funkci Playwright ~ Chromium pro stahování souborů
# Funkce spravena - stáhne všech 34 nalezených souborů

async def fetch_and_download_files(url, download_path):
    # Ensure the download directory exists
    if not os.path.exists(download_path):
        os.makedirs(download_path)
    
    async with async_playwright() as pw:
        browser = await pw.firefox.launch(headless=True)
        page = await browser.new_page()

        await page.goto(url)
        await page.wait_for_load_state('load')
        await page.wait_for_selector('a.download')

        download_links = await page.query_selector_all('a.download')
        total_links = len(download_links)
        print(f"Počet nalezených klikacích odkazů: {total_links}")

        downloaded_files = set()  # Track already downloaded files
        d_counts = 0
        file_counter = 1  # Start file numbering from 1

        if download_links:
            for link in download_links:
                href = await link.get_attribute('href')
                print(f"Klikáme na odkaz ke stažení: {href}")
                
                async with page.expect_download() as download_info:
                    await link.evaluate("(el) => el.click()")  # Force click
                
                download = await download_info.value
                file_name, file_extension = os.path.splitext(download.suggested_filename)
                
                # Ensure unique filenames
                new_file_name = f"{file_name}_{file_counter}{file_extension}"
                file_path = os.path.join(download_path, new_file_name)
                
                if new_file_name not in downloaded_files:  # Avoid duplicates
                    await download.save_as(file_path)
                    downloaded_files.add(new_file_name)
                    
                    if os.path.exists(file_path):  # Confirm file exists
                        d_counts += 1
                        print(f"Stažen soubor: {new_file_name}")
                        file_counter += 1  # Increment counter for the next file
                    else:
                        print(f"Soubor {new_file_name} se neobjevil!")  
                          
        print(f"Hotovo s {d_counts} stahováními")
        await browser.close()


In [24]:
# -------------------
# RUN download script
# -------------------
await fetch_and_download_files(url, download_path)

Počet nalezených klikacích odkazů: 34
Klikáme na odkaz ke stažení: /assets/cz/podnikani/zivnostenske-podnikani/statisticke-udaje-o-podnikatelich/2025/1/4---Pocty-podnikatelu-dle-obcanstvi-podnikajici-v-Ceske-republice-xlsx.xlsm
Stažen soubor: 4---Pocty-podnikatelu-dle-obcanstvi-podnikajici-v-Ceske-republice-xlsx_1.xlsm
Klikáme na odkaz ke stažení: /assets/cz/podnikani/zivnostenske-podnikani/statisticke-udaje-o-podnikatelich/2024/10/3---Pocty-podnikatelu-dle-obcanstvi-podnikajici-v-Ceske-republice.xlsx
Stažen soubor: 3---Pocty-podnikatelu-dle-obcanstvi-podnikajici-v-Ceske-republice_2.xlsx
Klikáme na odkaz ke stažení: /assets/cz/podnikani/zivnostenske-podnikani/statisticke-udaje-o-podnikatelich/2024/7/2---Pocty-podnikatelu-dle-obcanstvi-podnikajici-v-Ceske-republice_1.xlsx
Stažen soubor: 2---Pocty-podnikatelu-dle-obcanstvi-podnikajici-v-Ceske-republice_1_3.xlsx
Klikáme na odkaz ke stažení: /assets/cz/podnikani/zivnostenske-podnikani/statisticke-udaje-o-podnikatelich/2024/4/1---Pocet-podn

In [26]:
# Zjistit, kolik se stáhlo souborů a vypsat jména

folder_path = Path(download_path)

processed = 0
filenames = []
# Iterování nad soubory ve složce
for file in folder_path.iterdir():
    if file.is_file():  # Check if it's a file
        processed += 1
        filenames.append(file.name)
        #print(f"Processing file: {file.name}")
        # Skript automat tady

print(f"Ve složce je {processed} souborů.")
print(f"Toto jsou jejich jména: {filenames}")

Ve složce je 34 souborů.
Toto jsou jejich jména: ['2017---III---Q---Pocty-cizincu-podnikajicich-v-CR-dle--obcanstvi_30.xlsx', '4---Pocty-podnikatelu-dle-obcanstvi-podnikajicich-v-Ceske-republice_13.xlsx', '2---Pocty-podnikatelu-dle-obcanstvi-podnikajicich-v-Ceske-republice_7.xlsx', '4---Pocty-podnikatelu-dle-obcanstvi-podnikajicich-v-Ceske-republice_25.xlsx', '2---Pocty-podnikatelu-dle-obcanstvi-podnikajici-v-Ceske-republice_1_3.xlsx', '4---Pocty-podnikatelu-dle-obcanstvi-podnikajicich-v-Ceske-republice_5.xlsx', '4---Pocty-podnikatelu-dle-obcanstvi-podnikajicich-v-Ceske-republice_9.xlsx', '3---Pocty-podnikatelu-dle-obcanstvi-podnikajicich-v-Ceske-republice_10.xlsx', '1---Pocty-podnikatelu-dle-obcanstvi-podnikajicich-v-Ceske-republice_8.xlsx', '3---Pocty-podnikatelu-dle-obcanstvi-podnikajicich-v-Ceske-republice_6.xlsx', '1---Pocty-podnikatelu-dle-obcanstvi-podnikajicich-v-Ceske-republice_1_20.xlsx', '2---Pocty-podnikatelu-dle-obcanstvi-podnikajicich-v-Ceske-republice_11.xlsx', '4---Poct

In [32]:
# Zkusmo otevřít jeden .xlsx soubor a uložit ho jako .csv
file = "2016 Q4"
file_path = (f"{download_path}/{file}.xlsx")
df = pd.read_excel(file_path, engine='openpyxl')

# Save as CSV
csv_path = (f"{download_path}_CSV/{file}.csv")
df.to_csv(csv_path, index=False)
print(f"File converted to: {csv_path}")


File converted to: ./Download_CSV/2016 Q4.csv


In [199]:
# Otevřeme v pandasu a zahodíme 4 zbytečné řádky + resetujeme index

df = pd.read_csv(csv_path, header=None)  # Čte bez záhlaví
dff = df.iloc[4:].reset_index(drop=True)  # Odstraní první 4 řádky a resetuje index
dff.columns = dff.iloc[0]  # Nastaví 5. řádek (který je nyní první po odstranění) jako názvy sloupců
dff = dff.drop(0).reset_index(drop=True)  # Odstraní řádek použitý jako názvy sloupců

In [201]:
print(df.shape)
print(dff.shape)
print(dff.head(2))

(168, 19)
(163, 19)
0    Občanství  Kód Kód zn. Kód zn. Hlavní město Praha Středočeský kraj  \
0  Afghánistán  004      AF     AFG                 15                7   
1      Albánie  008      AL     ALB                 11                8   

0 Jihočeský kraj Plzeňský kraj Karlovarský kraj Ústecký kraj Liberecký kraj  \
0              0             0                1            6              2   
1              4             1                1            2              1   

0 Královéhradecký kraj Pardubický kraj Kraj Vysočina Jihomoravský kraj  \
0                    0               0             1                13   
1                    2               1             1                 2   

0 Olomoucký kraj Moravskoslezský kraj Zlínský kraj Součet  
0              4                    2            0     51  
1              1                    1            4     40  


In [204]:
# Základní přehled
print(f"Dataframe má tvar: {dff.shape}")
print(f"Indexy: {dff.index}")
print(f"Sloupce se jmenují: {dff.columns}")


Dataframe má tvar: (163, 19)
Indexy: RangeIndex(start=0, stop=163, step=1)
Sloupce se jmenují: Index(['Občanství', 'Kód', 'Kód zn.', 'Kód zn.', 'Hlavní město Praha',
       'Středočeský kraj', 'Jihočeský kraj', 'Plzeňský kraj',
       'Karlovarský kraj', 'Ústecký kraj', 'Liberecký kraj',
       'Královéhradecký kraj', 'Pardubický kraj', 'Kraj Vysočina',
       'Jihomoravský kraj', 'Olomoucký kraj', 'Moravskoslezský kraj',
       'Zlínský kraj', 'Součet'],
      dtype='object', name=0)


In [267]:
# Automatizace převodu celé složky .XLSX na správné .CSV
# Otevře, převede na .csv, odebere 4 první řádky, 
# resetuje index, uloží jako .csv

download_path = "./Download"
export_path = "./Download_CSV"


folder_path = Path(download_path)

'''
TEST - vypíše všechny soubory v Download složce

files = list(folder_path.iterdir())
for x in files:
    print(x)
'''

processed = 0
#filenames = []
# Prochází všechny soubory ve složce
for file in folder_path.iterdir():
    if file.is_file() and not file.name.startswith('.'):  # Kontrola, zda jde o soubor
        processed += 1
        #filenames.append(file.name)
        #print(f"Zpracovává se soubor: {file.name}")

        # Zkusmo otevřít jeden .xlsx soubor a uložit ho jako .csv
        file_path = (f"{download_path}/{file.name}")
        df = pd.read_excel(file_path, engine='openpyxl')

        newname = (file.name).replace(".xlsx", "") # odstranění .xlsx ze starého názvu
        csv_path = (f"{export_path}/{newname}.csv") # nový název pro uložení
        df.to_csv(csv_path, index=False)
        print(f"{processed} Soubor {csv_path} úspěšně převeden", end=" ")

        # Otevřeme v pandasu a zahodíme 4 zbytečné řádky + resetujeme index
        df = pd.read_csv(csv_path, header=None)  # Načte bez záhlaví
        dff = df.iloc[4:].reset_index(drop=True)  # Odstraní první 4 řádky a resetuje index
        dff.columns = dff.iloc[0]  # Nastaví 5. řádek (který je teď první po odstranění) jako názvy sloupců
        dff = dff.drop(0).reset_index(drop=True)  # Odstraní řádek použitý jako názvy sloupců

        # Uloží jako CSV
        dff.to_csv(csv_path, index=False)
        print(f"+ uložen", end="\n")

print(f"Ve složce je {processed} souborů.")
#print(f"Toto jsou jejich jména: {filenames}")


1 Soubor ./Download_CSV/2024_3.csv úspěšně převeden + uložen
2 Soubor ./Download_CSV/2019_1.csv úspěšně převeden + uložen
3 Soubor ./Download_CSV/2016_4.csv úspěšně převeden + uložen
4 Soubor ./Download_CSV/2017_1.csv úspěšně převeden + uložen
5 Soubor ./Download_CSV/2024_2.csv úspěšně převeden + uložen
6 Soubor ./Download_CSV/2018_4.csv úspěšně převeden + uložen
7 Soubor ./Download_CSV/2018_3.csv úspěšně převeden + uložen
8 Soubor ./Download_CSV/2018_2.csv úspěšně převeden + uložen
9 Soubor ./Download_CSV/2017_4.csv úspěšně převeden + uložen
10 Soubor ./Download_CSV/2018_1.csv úspěšně převeden + uložen
11 Soubor ./Download_CSV/2020_1.csv úspěšně převeden + uložen
12 Soubor ./Download_CSV/2017_2.csv úspěšně převeden + uložen
13 Soubor ./Download_CSV/2019_3.csv úspěšně převeden + uložen
14 Soubor ./Download_CSV/2024_1.csv úspěšně převeden + uložen
15 Soubor ./Download_CSV/2019_2.csv úspěšně převeden + uložen
16 Soubor ./Download_CSV/2017_3.csv úspěšně převeden + uložen
Ve složce je 16 s

In [256]:
# Definice proměnných

csv_file = "./Download_CSV/2016_2.csv"

suffix = csv_file.strip(".csv")
suffix = suffix[-1]

rok = csv_file.strip(".csv")
rok = rok[-6:-2]

In [257]:
# Otevřeme v pandasu jeden soubor a zkusíme řádky pojmenovat s příponou Q + číslo podle názvu souboru

df = pd.read_csv(csv_path)  # Read without headers
df["Rok"] = rok
df["Q"] = suffix
df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,Rok,Q
0,Občanství,Kód,Kód zn.,Kód zn.,Hlavní město Praha,Středočeský kraj,Jihočeský kraj,Plzeňský kraj,Karlovarský kraj,Ústecký kraj,...,Královéhradecký kraj,Pardubický kraj,Kraj Vysočina,Jihomoravský kraj,Olomoucký kraj,Moravskoslezský kraj,Zlínský kraj,Součet,2016,2
1,Afghánistán,004,AF,AFG,16,7,0,0,2,7,...,0,0,1,12,3,2,0,52,2016,2
2,Albánie,008,AL,ALB,14,7,4,1,1,3,...,2,1,1,2,1,1,4,43,2016,2
3,Alžírsko,012,DZ,DZA,92,19,4,0,1,7,...,1,1,1,8,3,2,1,142,2016,2
4,Angola,024,AO,AGO,10,2,2,0,0,2,...,1,1,0,6,3,1,2,33,2016,2


In [275]:
# Automat - přidá dva sloupce "rok" a "Q" jako kvartál

folder_path = Path("./Download_CSV")

for file in folder_path.iterdir():
    if file.is_file() and not file.name.startswith('.'):
        csv_file = file.name
        csv_path = (f"{folder_path}/{csv_file}")

        # Definice proměnných q, rok
        q = csv_file.strip(".csv")
        q = suffix[-1]

        rok = csv_file.strip(".csv")
        rok = rok[-6:-2]

        df = pd.read_csv(csv_path)
        df["Rok"] = rok
        df["Q"] = q

        df.to_csv(csv_path, index=False)

        print(f"Soubor {csv_file} byl změněn", end="\n")


Soubor 2016_2.csv byl změněn


In [276]:
df.head(5)

,Občanství,Kód,Kód zn.,Kód zn..1,Hlavní město Praha,Středočeský kraj,Jihočeský kraj,Plzeňský kraj,Karlovarský kraj,Ústecký kraj,...,Královéhradecký kraj,Pardubický kraj,Kraj Vysočina,Jihomoravský kraj,Olomoucký kraj,Moravskoslezský kraj,Zlínský kraj,Součet,Rok,Q
0,Afghánistán,4,AF,AFG,15,6,0,0,1,6,...,0,0,1,12,4,2,0,48,2016,2
1,Albánie,8,AL,ALB,10,8,4,1,1,2,...,2,1,0,2,2,1,3,38,2016,2
2,Alžírsko,12,DZ,DZA,99,21,3,0,1,9,...,1,1,0,8,2,3,0,150,2016,2
3,Angola,24,AO,AGO,9,2,2,0,0,3,...,1,1,0,6,3,4,2,35,2016,2
4,Ázerbájdžán,31,AZ,AZE,37,5,0,2,4,4,...,1,0,0,2,4,0,0,59,2016,2


In [294]:
df.shape
#df.columns

(162, 21)

In [295]:
df.columns

Index(['Občanství', 'Kód', 'Kód zn.', 'Kód zn..1', 'Hlavní město Praha',
       'Středočeský kraj', 'Jihočeský kraj', 'Plzeňský kraj',
       'Karlovarský kraj', 'Ústecký kraj', 'Liberecký kraj',
       'Královéhradecký kraj', 'Pardubický kraj', 'Kraj Vysočina',
       'Jihomoravský kraj', 'Olomoucký kraj', 'Moravskoslezský kraj',
       'Zlínský kraj', 'Součet', 'Rok', 'Q'],
      dtype='object')

In [307]:
for x in df["Občanství"].values:
    print(x)

Afghánistán
Albánie
Alžírsko
Angola
Ázerbájdžán
Argentina
Austrálie
Rakousko
Bangladéš
Arménie
Belgie
Bolívie
Bosna a Hercegovina
Brazílie
Britské Panenské ostrovy
Kosovo
Bulharsko
Myanmar
Bělorusko
Kambodža
Kamerun
Kanada
Šrí Lanka
Čad
Chile
Čína
Tchaj-wan
Vánoční ostrov
Kolumbie
Konžská republika
Demokratická republika
Kongo
Kostarika
Chorvatsko
Kuba
Kypr
Česko
Benin
Dánsko
Dominikánská republika
Ekvádor
Salvador
Etiopie
Eritrea
Estonsko
Finsko
Alandy
Francie
Francouzská Guyana
Gruzie
Gambie
Palestina
Německo
Ghana
Řecko
Guatemala
Guinea
Guyana
Honduras
Hongkong
Maďarsko
Island
Indie
Indonésie
Írán
Irák
Irsko
Izrael
Itálie
Pobřeží slonoviny
Jamajka
Japonsko
Kazachstán
Jordánsko
Keňa
Korejská lidově
demokratická republika
Korejská republika
Kuvajt
Kyrgyzstán
Laos
Libanon
Lotyšsko
Libérie
Libye
Litva
Lucembursko
Madagaskar
Malajsie
Mali
Malta
Mauritánie
Mauricius
Mexiko
Monako
Mongolsko
Moldavsko
Černá Hora
Maroko
Namibie
Nepál
Nizozemsko
Nizozemské Antily
Curacao
Vanuatu
Nový Zéland
N

In [33]:
import tabula

# Path to your PDF file
pdf_file_path = '2016 Q2.pdf'

# Path to save the CSV output
csv_file_path = '/Download_CSV/2016 Q2.csv'

# Use tabula to read the table from the PDF and convert it into a CSV
tabula.convert_into(pdf_file_path, csv_file_path, output_format='csv', pages='all')

print(f"PDF successfully converted to {csv_file_path}")

Failed to import jpype dependencies. Fallback to subprocess.
No module named 'jpype'
Error from tabula-java:
The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.





CalledProcessError: Command '['java', '-Djava.awt.headless=true', '-Dfile.encoding=UTF8', '-jar', '/Users/michaltrodler/.pyenv/versions/3.11.7/lib/python3.11/site-packages/tabula/tabula-1.0.5-jar-with-dependencies.jar', '--pages', 'all', '--guess', '--format', 'CSV', '--outfile', '/Download_CSV/2016 Q2.csv', '2016 Q2.pdf']' returned non-zero exit status 1.

In [273]:
''''
JEDNORÁZOVÉ SPRAVENÍ SOUBORU 2016_2.csv

file_path = 'Down/2016_2.csv'  # Replace with your file path
df = pd.read_csv(file_path, sep=';')

# Save the file with commas as the delimiter
output_path = 'fixed_file.csv'  # Replace with your desired output path
df.to_csv(output_path, index=False, sep=',')
'''

In [ ]:
'''
------------------------------ THE END ---------------------------------
'''